In [1]:
import requests
import lxml.html as lh
import pandas as pd

url='https://api.census.gov/data/2017/acs/acs5/variables.html'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]



[9, 1, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [2]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:%s' % (i,name))
    col.append((name,[]))
    

1:Name
2:Label
3:Concept
4:Required
5:Attributes
6:Limit
7:Predicate Type
8:Group
9:Values


In [3]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(6,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=9:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

[len(C) for (title,C) in col]

[25106, 25106, 25106, 25106, 25106, 25106, 25106, 25106, 25106]

In [4]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df.head()

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Values
0,B00001_001E,Estimate!!Total,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,not required,B00001_001EA,0,int,B00001,\n\t\t\t\t\t\tN/A\n\t\t\t\t\t
1,B00002_001E,Estimate!!Total,UNWEIGHTED SAMPLE HOUSING UNITS,not required,B00002_001EA,0,int,B00002,\n\t\t\t\t\t\tN/A\n\t\t\t\t\t
2,B01001_001E,Estimate!!Total,SEX BY AGE,not required,"B01001_001EA,\n B01001_001M,\n ...",0,(not a predicate),B01001,\n\t\t\t\t\t\tN/A\n\t\t\t\t\t
3,B01001_002E,Estimate!!Total!!Male,SEX BY AGE,not required,"B01001_002EA,\n B01001_002M,\n ...",0,int,B01001,\n\t\t\t\t\t\tN/A\n\t\t\t\t\t
4,B01001_003E,Estimate!!Total!!Male!!Under 5 years,SEX BY AGE,not required,"B01001_003EA,\n B01001_003M,\n ...",0,int,B01001,\n\t\t\t\t\t\tN/A\n\t\t\t\t\t
